In [32]:

import pandas as pd
import json
import os
from pathlib import Path
import PyPDF2
import numpy as np


food = pd.read_csv("data/csv/food.csv")
food_portion = pd.read_csv("data/csv/food_portion.csv")
nutrient = pd.read_csv("data/csv/nutrient.csv")



In [2]:
nutrient

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0
...,...,...,...,...,...
472,2064,Oligosaccharides,MG,NaN,2250.0
473,2065,Low Molecular Weight Dietary Fiber (LMWDF),G,293.4,1306.0
474,2068,Vitamin E,MG,959.0,7810.0
475,2067,Vitamin A,UG,960.0,7430.0


In [3]:
food

,fdc_id,data_type,description,food_category_id,publication_date
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13
1,1105905,branded_food,SWANSON BROTH BEEF,Herbs/Spices/Extracts,2020-11-13
2,1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,Prepared Soups,2020-11-13
3,1105907,branded_food,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,Prepared Soups,2020-11-13
4,1105898,experimental_food,Discrepancy between the Atwater factor predict...,NaN,2020-10-30
...,...,...,...,...,...
2064907,2731070,sub_sample_food,"Squash, yellow, raw",NaN,2024-01-18
2064908,2731071,sub_sample_food,"Squash, yellow, raw",NaN,2024-01-18
2064909,2731072,sub_sample_food,"Squash, yellow, raw",NaN,2024-01-18
2064910,2731073,sub_sample_food,"Squash, yellow, raw",NaN,2024-01-18


In [4]:
TARGET_NUTRIENTS = {
    "Energy (Atwater General Factors)": "calories",
    # "Energy (Atwater Specific Factors)": "calories",
    "Protein": "protein",
    "Carbohydrate, by difference": "carbs",
    "Total lipid (fat)": "fat",
    "Fiber, total dietary": "fiber",
    "Sugars, total": "sugar",
    "Calcium, Ca": "calcium",
    "Iron, Fe": "iron",
    "Potassium, K": "potassium",
    "Sodium, Na": "sodium",
    "Vitamin C, total ascorbic acid": "vitamin_c",
    "Vitamin A, RAE": "vitamin_a",
    "Vitamin D (D2 + D3)": "vitamin_d"
}
filtered_chunks = []
target_df = nutrient[nutrient["name"].isin(TARGET_NUTRIENTS.keys())]

NUTRIENT_ID_MAP = dict(zip(target_df["id"], target_df["name"]))
TARGET_NUTRIENT_IDS = list(NUTRIENT_ID_MAP.keys())
for k ,v in TARGET_NUTRIENTS.items():
    print(f"{k}")

Energy (Atwater General Factors)
Protein
Carbohydrate, by difference
Total lipid (fat)
Fiber, total dietary
Sugars, total
Calcium, Ca
Iron, Fe
Potassium, K
Sodium, Na
Vitamin C, total ascorbic acid
Vitamin A, RAE
Vitamin D (D2 + D3)


In [5]:
for chunk in pd.read_csv(
    "data/csv/food_nutrient.csv",
    usecols=["fdc_id", "nutrient_id", "amount"],
    chunksize=1_000_000,
    low_memory=False
):
    chunk = chunk[chunk["nutrient_id"].isin(TARGET_NUTRIENT_IDS)]
    filtered_chunks.append(chunk)

In [6]:
food_nutrient = pd.concat(filtered_chunks, ignore_index=True)

In [7]:
chunk

,fdc_id,nutrient_id,amount
1000020,174180,1004,0.00
1000021,174180,1106,0.00
1000023,174180,1079,0.00
1000032,174181,1087,714.00
1000036,174181,1114,12.50
...,...,...,...
1048520,174765,1114,0.00
1048525,174765,1089,1.86
1048540,174765,1162,0.00
1048546,174765,1087,18.00


In [8]:

TARGET_NUTRIENT_IDS

[2047, 1003, 1004, 1005, 1079, 1087, 1089, 1092, 1093, 1106, 1114, 1162]

In [9]:
NUTRIENT_ID_MAP 

{2047: 'Energy (Atwater General Factors)',
 1003: 'Protein',
 1004: 'Total lipid (fat)',
 1005: 'Carbohydrate, by difference',
 1079: 'Fiber, total dietary',
 1087: 'Calcium, Ca',
 1089: 'Iron, Fe',
 1092: 'Potassium, K',
 1093: 'Sodium, Na',
 1106: 'Vitamin A, RAE',
 1114: 'Vitamin D (D2 + D3)',
 1162: 'Vitamin C, total ascorbic acid'}

In [10]:
target_df

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
4,1003,Protein,G,203.0,600.0
5,1004,Total lipid (fat),G,204.0,800.0
6,1005,"Carbohydrate, by difference",G,205.0,1110.0
80,1079,"Fiber, total dietary",G,291.0,1200.0
88,1087,"Calcium, Ca",MG,301.0,5300.0
90,1089,"Iron, Fe",MG,303.0,5400.0
93,1092,"Potassium, K",MG,306.0,5700.0
94,1093,"Sodium, Na",MG,307.0,5800.0
107,1106,"Vitamin A, RAE",UG,320.0,7420.0


In [11]:
merged = food_nutrient.merge(food, on="fdc_id", how="left")

In [12]:
merged["nutrient"] = merged["nutrient_id"].map(NUTRIENT_ID_MAP)

In [13]:
merged

,fdc_id,nutrient_id,amount,data_type,description,food_category_id,publication_date,nutrient
0,1105904,1092,0.00,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13,"Potassium, K"
1,1105904,1089,0.00,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13,"Iron, Fe"
2,1105904,1162,0.00,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13,"Vitamin C, total ascorbic acid"
3,1105904,1087,0.00,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13,"Calcium, Ca"
4,1105904,1093,0.00,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13,"Sodium, Na"
...,...,...,...,...,...,...,...,...
316186,174765,1114,0.00,sr_legacy_food,"Beef, chuck, arm pot roast, separable lean onl...",13,2019-04-01,Vitamin D (D2 + D3)
316187,174765,1089,1.86,sr_legacy_food,"Beef, chuck, arm pot roast, separable lean onl...",13,2019-04-01,"Iron, Fe"
316188,174765,1162,0.00,sr_legacy_food,"Beef, chuck, arm pot roast, separable lean onl...",13,2019-04-01,"Vitamin C, total ascorbic acid"
316189,174765,1087,18.00,sr_legacy_food,"Beef, chuck, arm pot roast, separable lean onl...",13,2019-04-01,"Calcium, Ca"


In [14]:
final = (
    merged
    .pivot_table(
        index="description",
        columns="nutrient",
        values="amount",
        aggfunc="first"
    )
    .reset_index()
)

In [48]:

MAX_GRAMS = 100

for col in ["protein", "carbs", "fat", "fiber"]:
    # Values above MAX_GRAMS are likely mg → convert to g
    final.loc[final[col] > MAX_GRAMS, col] = final[col] / 1000


In [49]:
final.rename(columns={"description": "food_name"}, inplace=True)


In [50]:
final.rename(columns=TARGET_NUTRIENTS, inplace=True)


In [51]:
for col in ["protein", "carbs", "fat"]:
    final[col] = pd.to_numeric(final[col], errors="coerce")

In [52]:

final[["protein", "carbs", "fat"]] = final[["protein", "carbs", "fat"]].fillna(0)


In [53]:
final["calories"] = (
    final["protein"] * 4 +
    final["carbs"] * 4 +
    final["fat"] * 9
)


In [54]:
final.loc[final["calories"] <= 0, "calories"] = np.nan
final["calories"] = final["calories"].round(1)


In [55]:
final.isnull().sum()

nutrient
food_name        0
calcium       3896
carbs            0
fiber         4264
iron          3887
potassium    13191
protein          0
sodium         148
fat              0
vitamin_a    27482
vitamin_c    17875
vitamin_d    29079
calories      1236
dtype: int64

In [44]:
final

nutrient,food_name,calcium,carbs,fiber,iron,potassium,protein,sodium,fat,vitamin_a,vitamin_c,vitamin_d,calories
0,"""""FRIED RICE"""" PROTEIN PACK",48.0,6.32,0.0,1.05,161.0,12.63,253.0,10.53,NaN,NaN,NaN,170.6
1,"""""HOT AND SPICY"""" SWEET HEAT BARBECUE SAUCE, ""...",0.0,40.54,0.0,0.00,NaN,0.00,1432432.0,0.00,NaN,NaN,NaN,162.2
2,"""""MILK"""" & COOKIES VANILLA WITH CHOCOLATE CHIP...",0.0,33.33,0.8,0.98,0.0,2.27,95.0,13.64,NaN,NaN,NaN,265.2
3,"""""NO CHEESE"""" CHEESINESS GRAIN FREE PALEO PUFF...",11.0,64.29,3.6,0.00,46.0,3.57,679.0,21.43,NaN,NaN,NaN,464.3
4,"""""NO CHEESE"""" CHEESINESS ORGANIC POPCORN, """"NO...",16.0,56.00,16.0,4.00,252.0,8.00,640.0,14.00,NaN,NaN,NaN,382.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33929,"ZUCCHINI QUICK BREAD MIX, ZUCCHINI",19.0,81.48,0.0,0.00,67.0,7.41,593.0,0.00,NaN,NaN,NaN,355.6
33930,ZUCCHINI SQUASH,21.0,4.08,2.0,0.44,265.0,1.02,0.0,0.00,NaN,NaN,NaN,20.4
33931,ZUCCHINI STICKS,16.0,3.53,1.2,0.00,NaN,1.18,6.0,0.00,NaN,NaN,NaN,18.8
33932,"ZUCCHINI, CARROTS, CAULIFLOWER, LIMA BEANS, GR...",24.0,7.06,2.4,0.71,200.0,2.35,24.0,0.00,NaN,NaN,NaN,37.6


In [45]:
final.tail()

nutrient,food_name,calcium,carbs,fiber,iron,potassium,protein,sodium,fat,vitamin_a,vitamin_c,vitamin_d,calories
33929,"ZUCCHINI QUICK BREAD MIX, ZUCCHINI",19.0,81.48,0.0,0.00,67.0,7.41,593.0,0.0,NaN,NaN,NaN,355.6
33930,ZUCCHINI SQUASH,21.0,4.08,2.0,0.44,265.0,1.02,0.0,0.0,NaN,NaN,NaN,20.4
33931,ZUCCHINI STICKS,16.0,3.53,1.2,0.00,NaN,1.18,6.0,0.0,NaN,NaN,NaN,18.8
33932,"ZUCCHINI, CARROTS, CAULIFLOWER, LIMA BEANS, GR...",24.0,7.06,2.4,0.71,200.0,2.35,24.0,0.0,NaN,NaN,NaN,37.6
33933,Zwieback,20.0,74.20,2.5,0.60,305.0,10.10,227.0,9.7,16.0,5.3,0.0,424.5


In [56]:
final["calories"].describe()


count    32698.000000
mean       271.901725
std        184.104534
min          0.100000
25%        109.000000
50%        260.000000
75%        399.875000
max       1200.000000
Name: calories, dtype: float64

In [57]:
final[final["protein"] > 100][["food_name", "protein", "calories"]].head()


nutrient,food_name,protein,calories


In [58]:
final

nutrient,food_name,calcium,carbs,fiber,iron,potassium,protein,sodium,fat,vitamin_a,vitamin_c,vitamin_d,calories
0,"""""FRIED RICE"""" PROTEIN PACK",48.0,6.32,0.0,1.05,161.0,12.63,253.0,10.53,NaN,NaN,NaN,170.6
1,"""""HOT AND SPICY"""" SWEET HEAT BARBECUE SAUCE, ""...",0.0,40.54,0.0,0.00,NaN,0.00,1432432.0,0.00,NaN,NaN,NaN,162.2
2,"""""MILK"""" & COOKIES VANILLA WITH CHOCOLATE CHIP...",0.0,33.33,0.8,0.98,0.0,2.27,95.0,13.64,NaN,NaN,NaN,265.2
3,"""""NO CHEESE"""" CHEESINESS GRAIN FREE PALEO PUFF...",11.0,64.29,3.6,0.00,46.0,3.57,679.0,21.43,NaN,NaN,NaN,464.3
4,"""""NO CHEESE"""" CHEESINESS ORGANIC POPCORN, """"NO...",16.0,56.00,16.0,4.00,252.0,8.00,640.0,14.00,NaN,NaN,NaN,382.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33929,"ZUCCHINI QUICK BREAD MIX, ZUCCHINI",19.0,81.48,0.0,0.00,67.0,7.41,593.0,0.00,NaN,NaN,NaN,355.6
33930,ZUCCHINI SQUASH,21.0,4.08,2.0,0.44,265.0,1.02,0.0,0.00,NaN,NaN,NaN,20.4
33931,ZUCCHINI STICKS,16.0,3.53,1.2,0.00,NaN,1.18,6.0,0.00,NaN,NaN,NaN,18.8
33932,"ZUCCHINI, CARROTS, CAULIFLOWER, LIMA BEANS, GR...",24.0,7.06,2.4,0.71,200.0,2.35,24.0,0.00,NaN,NaN,NaN,37.6


In [59]:
import pandas as pd

food_portion = pd.read_csv(
    "data/csv/food_portion.csv",
    low_memory=False
)

food_portion = food_portion[
    ["fdc_id", "portion_description", "gram_weight"]
]

food_portion.head()


,fdc_id,portion_description,gram_weight
0,167512,NaN,34.0
1,167513,NaN,44.0
2,167514,NaN,28.0
3,167515,NaN,57.0
4,167516,NaN,39.0


In [61]:
food_portion

,fdc_id,portion_description,gram_weight
0,167512,NaN,34.0
1,167513,NaN,44.0
2,167514,NaN,28.0
3,167515,NaN,57.0
4,167516,NaN,39.0
...,...,...,...
47168,2710776,Quantity not specified,120.0
47169,2710777,Quantity not specified,248.0
47170,2710777,1 fl oz (no ice),31.0
47171,2710777,1 fl oz (with ice),23.0


In [62]:
food_portion.tail()

,fdc_id,portion_description,gram_weight
47168,2710776,Quantity not specified,120.0
47169,2710777,Quantity not specified,248.0
47170,2710777,1 fl oz (no ice),31.0
47171,2710777,1 fl oz (with ice),23.0
47172,2710777,1 fl oz (NFS),31.0


In [63]:
final.to_csv(
    "clean_food_nutrition_100g.csv",
    index=False
)
